In [ ]:
import tensorflow as tf

import pathlib
import matplotlib.pyplot as plt
import os
from PIL import Image
import numpy as np
import sys

import dotenv
dotenv.load_dotenv('/tf/science-de-la-donnee/.env')

# Changer les données dans le .enc pour le path
data_multi_path = os.environ.get('DATA_MULTI_PATH')
data_binary_path = os.environ.get('DATA_BINARY_PATH')
script_path = os.environ.get('SCRIPT_PATH')
weight_path = os.environ.get('WEIGHT_PATH')
models_path = os.environ.get('MODELS_PATH')

sys.path.insert(0, script_path)
sys.path.insert(1, models_path)

# scripts
import plotResults
import createTrainingData

# models
import resnet50
import homemade
# import vgg19
# import xception


In [ ]:
# GRRR Pawww

# model_choosen = 'resnet50'
model_choosen = 'homemade'
# model_choosen = 'vgg19'
# model_choosen = 'xception'

image_h, image_w = 150, 150
batch_size = 32
min_h = 25
min_w = 25

epochs = 15

# Si on veut faire une classification binaire ou multi
binary = True

if(binary):
    data_dir = data_binary_path
else:
    data_dir = data_multi_path


In [ ]:
# Alimentation des train/test set
train_set, test_set = tf.keras.preprocessing.image_dataset_from_directory(
  data_dir,
  validation_split=.2,
  subset='both',
  image_size=(image_h, image_w),
  seed=42,
  batch_size=batch_size
)

class_names = train_set.class_names
num_classes = len(class_names)

In [ ]:
match(model_choosen):
    case 'resnet50':
        model = resnet50.build((image_h, image_w), num_classes)
    case 'homemade':
        model = homemade.build((image_h, image_w), num_classes)
    # case 'vgg19':
    #     model = vgg19.build((image_h, image_w), num_classes)
    # case 'xception':
    #     model = xception.build((image_h, image_w), num_classes)

In [5]:
path = createTrainingData.create_training_data(weight_path, model, model_choosen, num_classes, image_h, image_w, batch_size)

checkpoint_path = path+"/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

weights_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path,
    verbose=1,
    save_weights_only=True,
    save_freq='epoch')

history = model.fit(train_set, epochs=epochs, validation_data=test_set, callbacks=[weights_callback])

Corrupt JPEG data: 419 extraneous bytes before marker 0xd9



Epoch 7: saving model to /tf/dataset/livrable1/weight/2023_10_03_13_11_25/cp-0007.ckpt
1035/1035 [==============================] - 221s 212ms/step - loss: 0.2270 - accuracy: 0.9015 - val_loss: 0.2255 - val_accuracy: 0.9060
Epoch 8/15
1035/1035 [==============================] - ETA: 0s - loss: 0.2159 - accuracy: 0.9092

2023-10-03 13:38:38.694296: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 419 extraneous bytes before marker 0xd9



Epoch 8: saving model to /tf/dataset/livrable1/weight/2023_10_03_13_11_25/cp-0008.ckpt
1035/1035 [==============================] - 208s 199ms/step - loss: 0.2159 - accuracy: 0.9092 - val_loss: 0.1970 - val_accuracy: 0.9193
Epoch 9/15
1035/1035 [==============================] - ETA: 0s - loss: 0.2065 - accuracy: 0.9131

2023-10-03 13:42:07.747750: W tensorflow/core/lib/png/png_io.cc:88] PNG warning: iCCP: known incorrect sRGB profile
Corrupt JPEG data: 419 extraneous bytes before marker 0xd9



Epoch 9: saving model to /tf/dataset/livrable1/weight/2023_10_03_13_11_25/cp-0009.ckpt
1035/1035 [==============================] - 210s 202ms/step - loss: 0.2065 - accuracy: 0.9131 - val_loss: 0.2304 - val_accuracy: 0.8984
Epoch 10/15


In [ ]:
epochs_range = range(epochs)
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']


plt.figure(figsize=(16, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

plt.savefig(path+"/metrics.png")

In [ ]:
# Load de la meilleur epoch
model_to_load_path = weight_path+"2023_10_03_11_26_35/cp-0004.ckpt"

model.load_weights(model_to_load_path)

In [ ]:
predictions = model.predict(test_set)

In [ ]:
loss, acc = model.evaluate(test_set)

In [ ]:
full_tests_images = []
full_tests_labels = []
for i,(batch_images, batch_labels) in enumerate(test_set):
    for im in batch_images:
        full_tests_images.append(im)
    for res in batch_labels:
        full_tests_labels.append(int(res))

In [ ]:
plotResults.plot_results(num_classes == 2, predictions, full_tests_labels, full_tests_images, class_names, path)

In [ ]:
tp = 0
fp = 0
fn = 0
tn = 0

for i in range(len(predictions)):
    predicted_label = class_names[np.argmax(predictions[i])]
    true_label = class_names[full_tests_labels[i]]

    if true_label == 'Photo' and predicted_label == 'Photo':
        tp += 1
    elif true_label == 'Photo' and predicted_label != 'Photo':
        fn += 1
    elif true_label != 'Photo' and predicted_label == 'Photo':
        fp += 1
    elif true_label != 'Photo' and predicted_label != 'Photo':
        tn += 1    

In [ ]:
print(tp, fp, fn, tn)

In [ ]:
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1 = 2 * (precision * recall) / (precision + recall)
accuracy = (tp + tn) / (tp + tn + fp + fn)
print(precision, recall, f1, accuracy)

In [ ]:
print("Confusion Matrix : ")
print(f"[{tp}] [{fp}]")
print(f"[{fn}] [{tn}]")